In [2]:
import numpy as np
import pandas as pd

In [336]:
df = pd.read_csv('results_cv.txt', sep=';', header = None)

In [337]:
#df = df.set_index(0)
df = df.rename(columns={0:'Dataset', 1:'Acc', 2: 'Time (s)'})

In [338]:
columns = ['Acc', 'Time', 'Acc_std']

In [339]:
df = pd.concat([df.groupby('Dataset').mean().round(3), df.groupby('Dataset')['Acc'].std().round(3)], axis = 1)
df.columns = columns
columns = ['Acc', 'Acc_std', 'Time']
df = df[columns]
df['Time'] = df['Time'].round(1)

In [340]:
df

,Acc,Acc_std,Time
Dataset,,,
0,0.988,0.001,893.5
1,0.954,0.003,357.6
2,0.939,0.005,194.9
3,0.924,0.010,122.3
4,0.978,0.003,31.6
5,0.953,0.005,38.6
6,1.000,0.000,47.2
7,0.966,0.000,32.4
8,0.837,0.023,8.6


In [341]:
path = 'data/ECG/'

In [342]:
X_test = np.load(path + 'X_test.npy')
X_train = np.load(path + 'X_train.npy')
# y_test = np.load(path + 'y_test.npy')
# y_train = np.load(path + 'y_train.npy')

In [334]:
df = pd.read_csv('results_train_cv.txt', sep=';', header = None)
columns = ['Dataset','fold', 'Acc', 'filters1', 'filters2', 'nodes1', 'dropout1']
df.columns = columns

df = df.where(df['Dataset'].isin([25])).dropna()

mean_df = df.groupby(np.arange(len(df)) // 5).mean()

selected_filters1, selected_filters2, nodes1, dropout1 = mean_df.iloc[mean_df['Acc'].idxmax(),[3,4,5,6]]

In [335]:
print('selected_filters1', int(selected_filters1) )
print('selected_filters2', int(selected_filters2) )
print('nodes1', int(nodes1) )
print('dropout1', dropout1 )

selected_filters1 128
selected_filters2 256
nodes1 8
dropout1 0.25


### Seq2Var + RF

In [51]:
df = pd.read_csv('results_seq2var_rf_cv.txt', sep=';', header = None)
columns = ['Dataset','Var', 'Seq2Var', 'bSeq2Var', 'Time', 'Params']
df.columns = columns
df = df.where(df['Dataset'].isin([14])).dropna()

In [52]:
df

,Dataset,Var,Seq2Var,bSeq2Var,Time,Params
21,14.0,0.977679,0.959821,0.888393,39.272101,"(True, 5, 'log2', 20)"
22,14.0,0.968750,0.968750,0.879464,39.274966,"(True, 5, 'log2', 20)"
23,14.0,0.968750,0.955357,0.892857,39.273865,"(True, 5, 'log2', 20)"


In [53]:
mean_df = df.iloc[:,:-1].groupby('Dataset').mean()
df = pd.concat([df.iloc[:,:-1].groupby('Dataset').mean().round(3), df.iloc[:,:-1].groupby('Dataset')[['Var','Seq2Var','bSeq2Var']].std().round(3)], axis = 1)
columns = ['Acc_Var', 'Acc_Seq2Var', 'Acc_bSeq2Var','Time','std_var', 'std_seq2var', 'std_bseq2var']
df.columns = columns
columns = ['Acc_Var', 'std_var', 'Acc_Seq2Var', 'std_seq2var', 'Acc_bSeq2Var', 'std_bseq2var', 'Time']
df = df[columns]
df['Time'] = (df['Time']/2).round(1)

In [54]:
df

,Acc_Var,std_var,Acc_Seq2Var,std_seq2var,Acc_bSeq2Var,std_bseq2var,Time
Dataset,,,,,,,
14.0,0.972,0.005,0.961,0.007,0.887,0.007,19.6


### Raw / multiBEATS datasets + RF

In [269]:
df = pd.read_csv('results_rf.txt', sep=';', header = None)

In [270]:
columns = ['Dataset','Acc', 'Time', 'Params']
df.columns = columns
df = df.where(df['Dataset'].isin([3])).dropna()

In [271]:
df

,Dataset,Acc,Time,Params
9,3.0,0.806316,2.139614,"(True, 15, 'sqrt', 100)"
10,3.0,0.823860,2.132782,"(True, 15, 'sqrt', 100)"
11,3.0,0.823158,2.154598,"(True, 15, 'sqrt', 100)"


In [272]:
mean_df = df.iloc[:,:-1].groupby('Dataset').mean()
df = pd.concat([df.iloc[:,:-1].groupby('Dataset').mean().round(3), df.iloc[:,:-1].groupby('Dataset')[['Acc']].std().round(3)], axis = 1)
columns = ['Acc','Time','std_acc']
df.columns = columns
columns = ['Acc','std_acc','Time']
df = df[columns]
df['Time'] = (df['Time']).round(2)

In [273]:
df

,Acc,std_acc,Time
Dataset,,,
3.0,0.818,0.01,2.14


### Seq2Var + LSTM-FCN

In [333]:
df = pd.read_csv('results_seq2var_lstmfcn.txt', sep=';', header = None)
columns = ['Dataset','Acc', 'Time', 'Method']
df.columns = columns
df = df.where(df['Dataset'].isin([14])).dropna()
df = pd.concat([df.groupby(['Dataset','Method']).mean().round(3), df.groupby(['Dataset','Method'])[['Acc']].std().round(3)], axis = 1)
columns = ['Acc', 'Time', 'std_acc']
df.columns = columns
columns = ['Acc','std_acc', 'Time']

df['Time'] = ((df['Time']) - df['Time'].loc[(14.0, 'VAR')]/2).round(1)
df = df[columns]
df

Acc  std_acc  Time
Dataset Method                        
14.0    Seq2VAR   0.946    0.028  56.1
        VAR       0.512    0.545  56.3
        bSeq2VAR  0.924    0.007  54.8